# UFC Fight Analysis

<img src = 'images/touch_gloves.jpg'>

 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Todo" data-toc-modified-id="Todo-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Todo</a></span></li><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Introduction" data-toc-modified-id="Introduction-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Background" data-toc-modified-id="Background-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Background</a></span></li><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Data" data-toc-modified-id="Data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data</a></span></li><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Variables" data-toc-modified-id="Variables-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Variables</a></span></li><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Description" data-toc-modified-id="Description-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Description</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Some-Notes-to-observe" data-toc-modified-id="Some-Notes-to-observe-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Some Notes to observe</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Data-Cleaning" data-toc-modified-id="Data-Cleaning-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Data Cleaning</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Cleaning-and-Editing-Name-Column" data-toc-modified-id="Cleaning-and-Editing-Name-Column-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Cleaning and Editing Name Column</a></span></li><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Extracting-Country-from-B_Location-and-R_Location" data-toc-modified-id="Extracting-Country-from-B_Location-and-R_Location-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Extracting Country from B_Location and R_Location</a></span></li><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Converting-Strings-to-Categories" data-toc-modified-id="Converting-Strings-to-Categories-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Converting Strings to Categories</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Data-Correlation" data-toc-modified-id="Data-Correlation-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Data Correlation</a></span></li><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Modeling" data-toc-modified-id="Modeling-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Modeling</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Trying-to-improve-results-by-dividing-features" data-toc-modified-id="Trying-to-improve-results-by-dividing-features-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Trying to improve results by dividing features</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Dropping-Round-4-and-Round-5-since-most-fights-are-3-round-Max." data-toc-modified-id="Dropping-Round-4-and-Round-5-since-most-fights-are-3-round-Max.-9.1.1"><span class="toc-item-num">9.1.1&nbsp;&nbsp;</span>Dropping Round 4 and Round 5 since most fights are 3 round Max.</a></span></li><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Dropping-5-round-fights-entirely" data-toc-modified-id="Dropping-5-round-fights-entirely-9.1.2"><span class="toc-item-num">9.1.2&nbsp;&nbsp;</span>Dropping 5 round fights entirely</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Dropping-First-Fights" data-toc-modified-id="Dropping-First-Fights-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Dropping First Fights</a></span></li><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Aggregate-Round-Stats" data-toc-modified-id="Aggregate-Round-Stats-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>Aggregate Round Stats</a></span></li><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Comparing-Red-to-Blue" data-toc-modified-id="Comparing-Red-to-Blue-9.4"><span class="toc-item-num">9.4&nbsp;&nbsp;</span>Comparing Red to Blue</a></span></li><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Reducing-Features" data-toc-modified-id="Reducing-Features-9.5"><span class="toc-item-num">9.5&nbsp;&nbsp;</span>Reducing Features</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Conclusion" data-toc-modified-id="Conclusion-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Conclusion</a></span></li><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Stretch-Goals" data-toc-modified-id="Stretch-Goals-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Stretch Goals</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Rewrite-and-re-run-scraper-to-pull-data-from-earlier-years" data-toc-modified-id="Rewrite-and-re-run-scraper-to-pull-data-from-earlier-years-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>Rewrite and re-run scraper to pull data from earlier years</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Scraper-Rewrite" data-toc-modified-id="Scraper-Rewrite-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Scraper Rewrite</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Fight-URL-Scraper" data-toc-modified-id="Fight-URL-Scraper-12.1"><span class="toc-item-num">12.1&nbsp;&nbsp;</span>Fight URL Scraper</a></span></li><li><span><a href="http://localhost:8888/notebooks/ufc_fight_analysis/ufc_analysis.ipynb#Fight-Scraper" data-toc-modified-id="Fight-Scraper-12.2"><span class="toc-item-num">12.2&nbsp;&nbsp;</span>Fight Scraper</a></span></li></ul></li></ul></div>


## Todo


1. Create percentages column for each landed / attempted statistic
2. Aggregate all data so it's ongoing
  1. Sum each fighter's data to make it possible to compare fighters in a reasonable way
  2. One idea is to subtract two fighters' non-ratio stats and use them as labels
  3. You can still use ratio stats as labels
3. Rewrite models to account for the unbalanced data.
  1. Most fights are won by decision, so that should be accounted for
4. Add visuals

I'm just spitballing but I think the fighter with the most strikes landed will usually win. We can test this by turning each fight into differences. We will subtract one fighter's stats from the other, including strikes, submissions attempted, KO wins, KO losses, sub wins, etc. Then, we'll feed the data into the algorithm and assess the results. Currently, the algorithm is just looking at each fight's stats and judging the winner. That's not very realistic or useful tbh...

This will be a cool project, even if the results don't turn out the way I expected. We'll work on this in a couple stages:

1. Clean up data. Check for:
  1. Misspellings
  2. Logical errors (strikes landed > strikes attempted, etc)
  3. Incorrect data types
  4. Missing data
  5. Convert strings to categories? Not many categorical vars, so we'll leave this as a stretch goal.
2. Create df of running tallies. We'll have to put more thought into it, but searching for previous columns with that name and adding them together.
3. Only include non-debut fights? There will be no data for that fighter, which may skew results but I'm also curious about the outcome. I might include it anyway to see if there are any patterns.
4. In each fight, subtract one fighter's stats from the other

## Introduction

## Background

## Data

This data was scraped from www.fightmetrics.com with Beautiful Soup in two stages. The first script compiled a list of event urls and wrote them to a CSV. The second script crawled through those urls and extracted all available fight data into a pandas dataframe. There are 29 columns and 8774 records. The first (unsexy) step in the process will involve cleaning the dataset.

In [1]:
# Imports and Helper Functions
# data Analysis
import pandas as pd
import numpy as np
import random as rng
from pprint import pprint

# Web Scraping
import json
import codecs
import csv
import datetime
from bs4 import BeautifulSoup
import requests
import cStringIO
import pprint

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#SciKit Learn Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier


from sklearn.metrics import accuracy_score, classification_report,confusion_matrix

from subprocess import check_output
print check_output(["ls", "data"]).decode("utf8")
fight_data = pd.read_csv("data/fights.csv")
# Any results you write to the current directory are saved as output.

# Noteboook Functionality
from IPython.core.interactiveshell import InteractiveShell # All statements are printed to output
InteractiveShell.ast_node_interactivity = "all"

data.csv
data.csv.zip
fights.csv



In [2]:
# Creates a copy of the dataframe for testing
fight_df = pd.DataFrame(fight_data.copy())

# Rearranges the columns so the first name, last name and win/loss appear first
cols = fight_df.columns.tolist()
cols = cols[6:7] + cols[15:17] + cols[28:29] + cols[:6] + cols[7:15] + cols[17:28]
fight_df = fight_df[cols]
cols

# Sets the index as the fight_id
# fight_df.set_index('fight_id', inplace=True)

fight_df.head()
fight_df.to_csv('fights.csv', index=False)

['fight_id',
 'name_first',
 'name_last',
 'win/loss',
 'body',
 'body_attempts',
 'clinch',
 'clinch_attempts',
 'distance',
 'distance_attempts',
 'ground',
 'ground_attempts',
 'head',
 'head_attempts',
 'kd',
 'leg',
 'leg_attempts',
 'method',
 'pass',
 'referee',
 'reversals',
 'round',
 'sig_attempts',
 'sig_strikes',
 'strike_attempts',
 'strikes',
 'sub_attempts',
 'takedowns',
 'td_attempts']

,fight_id,name_first,name_last,win/loss,body,body_attempts,clinch,clinch_attempts,distance,distance_attempts,...,referee,reversals,round,sig_attempts,sig_strikes,strike_attempts,strikes,sub_attempts,takedowns,td_attempts
0,0,Ovince,Saint,1,0,0,0,0,0,0,...,Leon Roberts,0,1,0,0,3,3,1,0,0
1,0,Preux,Yushin,0,0,0,0,0,0,0,...,Leon Roberts,0,1,0,0,0,0,1,0,1
2,1,Claudia,Gadelha,1,1,4,9,15,30,71,...,John Sharp,0,3,91,42,96,47,2,0,7
3,1,Jessica,Andrade,0,50,58,18,25,45,107,...,John Sharp,0,3,241,141,355,242,0,4,7
4,2,Takanori,Gomi,1,1,1,0,0,2,3,...,Steve Perceval,0,1,3,2,3,2,0,0,0


## Variables



In [3]:
fight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8774 entries, 0 to 8773
Data columns (total 29 columns):
fight_id             8774 non-null int64
name_first           8774 non-null object
name_last            8774 non-null object
win/loss             8774 non-null int64
body                 8774 non-null int64
body_attempts        8774 non-null int64
clinch               8774 non-null int64
clinch_attempts      8774 non-null int64
distance             8774 non-null int64
distance_attempts    8774 non-null int64
ground               8774 non-null int64
ground_attempts      8774 non-null int64
head                 8774 non-null int64
head_attempts        8774 non-null int64
kd                   8774 non-null int64
leg                  8774 non-null int64
leg_attempts         8774 non-null int64
method               8774 non-null object
pass                 8774 non-null int64
referee              8738 non-null object
reversals            8774 non-null int64
round                8774 no

## Description
From this, we can see that we have a total of 29 Columns and one dependent variable. The columns themselves have 4 object types (First Name, Last Name, Method and Referee), and 25 Float types. This however does not give us a complete picture of the data, so we'll using a few other pandas functions to get a better glimpse. 

In [4]:
fight_df.describe()

,fight_id,win/loss,body,body_attempts,clinch,clinch_attempts,distance,distance_attempts,ground,ground_attempts,...,pass,reversals,round,sig_attempts,sig_strikes,strike_attempts,strikes,sub_attempts,takedowns,td_attempts
count,8774.000000,8774.000000,8774.000000,8774.000000,8774.000000,8774.000000,8774.000000,8774.000000,8774.000000,8774.000000,...,8774.000000,8774.000000,8774.000000,8774.000000,8774.000000,8774.000000,8774.000000,8774.000000,8774.000000,8774.000000
mean,2193.618874,0.500000,5.961591,8.703556,5.236152,7.848986,19.463415,54.518578,5.206633,7.829154,...,1.140301,0.140187,2.264418,70.196718,29.906200,91.531571,48.951333,0.456234,1.098017,2.844883
std,1267.532948,0.500028,7.138084,10.033876,7.590997,10.711031,22.495543,58.464204,9.037458,13.385800,...,2.056766,0.427533,1.008035,62.533774,26.796296,72.518595,41.855437,0.911112,1.695560,3.691603
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1096.250000,0.000000,1.000000,2.000000,0.000000,1.000000,3.000000,11.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,22.000000,9.000000,31.000000,16.000000,0.000000,0.000000,0.000000
50%,2193.000000,0.500000,4.000000,5.000000,2.000000,4.000000,12.000000,34.000000,1.000000,2.000000,...,0.000000,0.000000,3.000000,54.000000,23.000000,78.000000,40.000000,0.000000,0.000000,1.000000
75%,3289.750000,1.000000,9.000000,13.000000,7.000000,11.000000,27.000000,79.000000,7.000000,10.000000,...,1.000000,0.000000,3.000000,101.000000,44.000000,136.000000,71.000000,1.000000,2.000000,4.000000
max,4412.000000,1.000000,92.000000,101.000000,95.000000,115.000000,228.000000,447.000000,100.000000,136.000000,...,26.000000,5.000000,5.000000,495.000000,238.000000,497.000000,361.000000,10.000000,21.000000,33.000000


In [5]:
fight_df.describe(include=['O'])

,name_first,name_last,method,referee
count,8774,8774,8774,8738
unique,900,1483,8,155
top,Chris,Silva,Decision,Herb Dean
freq,147,102,3784,1306


In [6]:
fight_df['method'].value_counts()

Decision      3784
KO/TKO        2790
Submission    1850
Overturned     136
TKO            130
Could           52
DQ              24
Other            8
Name: method, dtype: int64

### Some Notes to observe

1. Most fights are won by decision. 
2. Fighter names are split up into first and last, which will have to be changed.
3. There are at least 36 records (18 fights) where the referee data is missing.

## Data Cleaning

The first step in data cleaning is to remove obvious outliers and columns that will not contribute to the model. One starting point is narrowing down the fights to just wins and losses, excluding no contests and draws. No contests are nearly impossible to predict as are draws, so it doesn't make sense to account for them. Here's a list of ideas so far:

1. Draws or no contest ['winner']
2. Blue and red ID
3. Blue and red Name
4. Blue and red Name
5. Date

The data is currently not in "tidy" format, so I may consider reshaping it. One obvious hint that this is the case is that every row has a round_4 and round_5 column, even though not all fights go to the last round. I'll see what I can do in terms of reshaping the data and look into whether it makes sense to do so. I wonder if there's a dplyr and tidyr package for Python? The R equivalents are pretty robust, so I'll probably start there. Here's an ongoing list of ideas that may be worth pursuing:

1. Rearrange data to get rid of empty Round4 and Round5 data
2. Separate city and country (Brazil vs RioDijanero Brazil and USA vs Stockton, California USA may add more predictive power)
3. Change landed and attempted data into percentages. This may help make better comparisons across fighters making debuts vs fighters with established records.
4. Add a column for submission wins, KO wins, etc

I'm debating whether to turn the database into a database of fighters instead of a database of fights. One reason is that you have the accumulated statistics on each fighter's history going into each event. If you have Jon Jones matching up against DC, it may be useful to ask, "What is it about Jon Jones' record that makes him a favorite? This may require a lot or rearranging, so it may be worth it to think this through a bit first.

If I were to go this route, I would reset each fighter's stats and only add stats that came prior to each fight. For example, if Frankie Edgar fought in 2013 against Anderson Silva and then fought again against Matt Hughes in 2014, I'd want their fighter record inputs to be different. I think this may be a more comprehensive way of looking at the data and would align more closely to real-world applications. Here's a rough sketch of what the process might look like:

1. Sort the dataframe by date
2. Add each fighter's stats cumulatively based on prior fights
  1. Figure out a way to accomplish this even though red is on one side and blue is on the other
  2. Maybe put fighters on just red or blue? Might be impossible..
  3. Maybe just account for this in the code? If B_Name == 'X' do this or if R_Name == 'X' do that
3. Add columns for kicks taken, punches taken, etc?
  1. One disadvantage is that this would mean nearly doubling the number of variables in a dataset that is already massive. This will hurt some machine learning models. Actually, this could lead to more variables than observations which would be no bueno, although it would still be possible to pare it down afterwards. Let's omit this for now and come back to it later if necessary.
4. Consider consolidating round data. Why do you need 5 rounds of data for each fighter? Consider building a granular model and an aggragated model.

In [7]:
fight_df.head()

,fight_id,name_first,name_last,win/loss,body,body_attempts,clinch,clinch_attempts,distance,distance_attempts,...,referee,reversals,round,sig_attempts,sig_strikes,strike_attempts,strikes,sub_attempts,takedowns,td_attempts
0,0,Ovince,Saint,1,0,0,0,0,0,0,...,Leon Roberts,0,1,0,0,3,3,1,0,0
1,0,Preux,Yushin,0,0,0,0,0,0,0,...,Leon Roberts,0,1,0,0,0,0,1,0,1
2,1,Claudia,Gadelha,1,1,4,9,15,30,71,...,John Sharp,0,3,91,42,96,47,2,0,7
3,1,Jessica,Andrade,0,50,58,18,25,45,107,...,John Sharp,0,3,241,141,355,242,0,4,7
4,2,Takanori,Gomi,1,1,1,0,0,2,3,...,Steve Perceval,0,1,3,2,3,2,0,0,0


In [8]:
fight_df.tail()

,fight_id,name_first,name_last,win/loss,body,body_attempts,clinch,clinch_attempts,distance,distance_attempts,...,referee,reversals,round,sig_attempts,sig_strikes,strike_attempts,strikes,sub_attempts,takedowns,td_attempts
8769,4410,Art,Jimmerson,0,0,0,0,0,0,0,...,Joao Barreto,0,1,0,0,0,0,0,0,0
8770,4411,Kevin,Rosier,1,3,4,4,9,4,10,...,Joao Barreto,0,1,27,15,53,38,0,0,0
8771,4411,Zane,Frazier,0,3,6,10,19,0,7,...,Joao Barreto,0,1,28,12,29,13,0,0,0
8772,4412,Gerard,Gordeau,1,0,0,0,0,1,3,...,Joao Barreto,0,1,5,3,5,3,0,0,0
8773,4412,Teila,Tuli,0,0,0,0,0,0,1,...,Joao Barreto,0,1,1,0,1,0,0,0,1


The head() and tail() functions give us a snapshot of the dataset's values. Not all rows and columns are represented, but it gives just enough to get a sense of how the data is organized. Right off the bat I can see the data is organized in reverse-chronological order. Art Jimmerson fought in the first UFC event and I still remember him because he was the only fighter to come to the cage wearing one boxing glove. It was a crazy time... I'll reverse the dataframe so it's in chronological order and re-index the records. 

I'm surprised by the lack of NaN values in this dataset. Apparently, Fightmetrics is very organized! I'll have to crawl deep to see if there are any inconsistencies.

In [9]:
fight_df['fight_id'] = abs(4412 - fight_df['fight_id'])
fight_df = fight_df[::-1]
index = list(range(8774))
fight_df.index = index

In [10]:
fight_df.head()

,fight_id,name_first,name_last,win/loss,body,body_attempts,clinch,clinch_attempts,distance,distance_attempts,...,referee,reversals,round,sig_attempts,sig_strikes,strike_attempts,strikes,sub_attempts,takedowns,td_attempts
0,0,Teila,Tuli,0,0,0,0,0,0,1,...,Joao Barreto,0,1,1,0,1,0,0,0,1
1,0,Gerard,Gordeau,1,0,0,0,0,1,3,...,Joao Barreto,0,1,5,3,5,3,0,0,0
2,1,Zane,Frazier,0,3,6,10,19,0,7,...,Joao Barreto,0,1,28,12,29,13,0,0,0
3,1,Kevin,Rosier,1,3,4,4,9,4,10,...,Joao Barreto,0,1,27,15,53,38,0,0,0
4,2,Art,Jimmerson,0,0,0,0,0,0,0,...,Joao Barreto,0,1,0,0,0,0,0,0,0


### Cleaning and Editing Name Column

The fighter names are split into first and last, which doesn't seem necessary. I'll combine them into one column, but first I want to check to make sure there are no missing values.

In [11]:
fight_df[fight_df['name_first'].str.isalpha()==False]

,fight_id,name_first,name_last,win/loss,body,body_attempts,clinch,clinch_attempts,distance,distance_attempts,...,referee,reversals,round,sig_attempts,sig_strikes,strike_attempts,strikes,sub_attempts,takedowns,td_attempts
4439,2245,Nah-Shon,Burrell,1,16,20,13,13,80,171,...,Jason Herzog,1,3,216,121,222,126,1,0,0
4638,2345,Nah-Shon,Burrell,0,29,38,32,47,24,74,...,Steve Mazzagatti,0,3,121,56,144,78,0,0,1
8000,4026,Jr.,Tyson,0,1,1,0,0,4,7,...,Herb Dean,0,1,7,4,21,15,1,2,2
8602,4327,Jr.,Paul,0,3,4,0,0,17,29,...,Leon Roberts,0,1,29,17,29,17,0,0,1
8690,4371,Abdul-Kerim,Edilov,0,16,17,6,12,9,22,...,Kevin Sataki,0,2,125,77,210,150,1,2,2


In [12]:
fight_df[fight_df['name_last'].str.isalpha()==False]

,fight_id,name_first,name_last,win/loss,body,body_attempts,clinch,clinch_attempts,distance,distance_attempts,...,referee,reversals,round,sig_attempts,sig_strikes,strike_attempts,strikes,sub_attempts,takedowns,td_attempts
791,421,Georges,St-Pierre,1,14,15,0,0,7,15,...,Steve Mazzagatti,2,3,80,46,177,138,0,6,8
821,436,Georges,St-Pierre,1,2,2,2,3,13,16,...,Steve Mazzagatti,0,1,33,24,46,37,0,0,1
858,455,Georges,St-Pierre,0,2,2,1,1,5,16,...,John McCarthy,0,1,17,6,24,12,0,1,1
907,479,Georges,St-Pierre,1,16,17,6,7,13,26,...,John McCarthy,1,3,94,64,148,117,4,5,5
955,503,Georges,St-Pierre,1,0,0,0,0,4,6,...,John McCarthy,0,1,32,25,44,37,2,1,1
1019,535,Georges,St-Pierre,1,11,11,7,7,34,72,...,Herb Dean,0,2,111,58,132,78,0,3,4
1071,561,Georges,St-Pierre,1,7,10,7,18,46,108,...,Herb Dean,0,3,135,59,212,135,0,4,5
1189,620,Jake,O'Brien,1,0,0,0,0,0,3,...,John McCarthy,0,2,52,25,116,85,0,3,3
1289,670,Jake,O'Brien,1,1,1,0,0,0,7,...,Steve Mazzagatti,0,3,31,13,132,106,0,10,13
1305,678,Georges,St-Pierre,1,3,5,4,5,33,74,...,John McCarthy,0,2,92,45,96,49,0,1,1


Looks good, all names are in the normal range. Now, to combine the first and last names for each figher.

In [13]:
fight_df.loc[:,'name'] = fight_df['name_first'] + ' ' + fight_df['name_last']
fight_df['name'].head()
del fight_df['name_first']
del fight_df['name_last']

0        Teila Tuli
1    Gerard Gordeau
2      Zane Frazier
3      Kevin Rosier
4     Art Jimmerson
Name: name, dtype: object

### Extracting Country from B_Location and R_Location

I want to extract each figher's country location. Currently there are 438 unique locations in the dataset. This is too many categories to get a sense of where fighters are training. There may also be some correlation between a fighter's location and his or her record. The first thing I did was exclude any fighters with null values for their current location. Thankfully, there were only 8 such cases. Then I cleaned up some of the mispellings / duplicate entries (USA and United States were consolidated into just USA). Finally, I replaced all [City Country] values to [Country] values.

In [ ]:
data['B_Location'].value_counts()[:10]

In [ ]:
data = data[data['winby'].notnull()]
loc_data = data[(data['B_Location'].notnull()) & data['R_Location'].notnull()]
data = data[data['winner'] != 'no contest']
#data = data.reset_index()

In [ ]:
count = loc_data['B_Location'].str.split().apply(len).value_counts()
count.index = count.index.astype(str) + ' words:'
count.sort_index(inplace=True)
count

In [ ]:
len(loc_data)
len(data)

In [ ]:
locations = ['B_Location', 'R_Location']
countries = ['Japan','Singapore']
for location in locations:
    for country in countries:
        loc_data.loc[(loc_data[location] == country), location] = 'Unknown '+country

In [ ]:
# This script cleans up mispellings and extracts country info from city country columns
locations = ['B_Location', 'R_Location']
new_cols = ['R_Country_Location', 'B_Country_Location']

c_dict = {
    'United': 'USA',
    'Brasil': 'Brazil',
    'Englad': 'England',
    'Czech': 'CzechRepublic',
    'Moldova': 'Moldova',
}

post_c_dict = {
    'Africa': 'South Africa',
    'CzechRepublic': 'Czech Republic',
    'PAN': 'Panama',
    'Zealand': 'New Zealand'
}
countries = ['']
for location in locations:
    for k,v in c_dict.items():
        loc_data.loc[loc_data[location].str.contains(k), location] = v

for new_col, location in zip(new_cols, locations):
    loc_data[new_col] = loc_data[location].str.split().str[-1]
for col in new_cols:
    for k,v in post_c_dict.items():
        loc_data.loc[loc_data[col].str.contains(k), col] = v

for location, col in zip(locations, new_cols):
    loc_data[location] = loc_data[col]
    del loc_data[col]
sorted(set(loc_data['B_Location'].values.tolist()))

In [ ]:
loc_data['B_Location'].value_counts()

In [ ]:
data.fillna(value=0,inplace=True)

In [ ]:
data.tail()

In [ ]:
dropdata = data.drop(['B_ID','B_Name','R_ID','R_Name','Date'],axis=1)
dropdata.rename(columns={'BPrev':'B__Prev',
                         'RPrev':'R__Prev',
                         'B_Age':'B__Age',
                         'B_Height':'B__Height',
                         'B_Weight':'B__Weight',
                         'R_Age':'R__Age',
                         'R_Height':'R__Height',
                         'R_Weight':'R__Weight',
                         'BStreak':'B__Streak',
                         'RStreak': 'R__Streak'},inplace=True)
dropdata.describe()

In [ ]:
dropdata.describe(include=['O'])

In [ ]:
dropdata.describe(include=[np.int])

### Converting Strings to Categories

Next we need to convert our object types of columns into categorical columns. This is done to sort by logical order

In [ ]:
objecttypes = list(dropdata.select_dtypes(include=['O']).columns)
for col in objecttypes:
    dropdata[col] = dropdata[col].astype('category')

In [ ]:
cat_columns = dropdata.select_dtypes(['category']).columns
dropdata[cat_columns] = dropdata[cat_columns].apply(lambda x: x.cat.codes)
dropdata.info()
dropdata.tail()

## Data Correlation
While it would otherwise be normal practise to draw a heatmap or correlation matrix of our data to look for linear relationships, this is highly illogical due to the sheer number of features we're currently looking at.  Instead we examine the n largest correlations with our dependent variable (winner) to look for linear relationships - as you can see from the plot below, the relationships (if they exist) are highly non-linear. This suggests that alterations to our Data Set are required. 

Further, it seems that Round 4 statistics for the Red Fighter are the more correlating, this suggests that a split or delta sort of data set should produce better results. 

In [ ]:
# Basic Correlation Matrix
# corrmat = data.corr()
# f, ax = plt.subplots(figsize=(12, 9))
# sns.heatmap(corrmat, vmax=.8, square=True);

In [ ]:
# Subset Correlation Matrix
k = 10 #number of variables for heatmap
corrmat = dropdata.corr()
cols = corrmat.nlargest(k, 'winner')['winner'].index
cm = np.corrcoef(dropdata[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

## Modeling
We're evaluating the following models

1. Perceptron
2. Random Forests
3. Decision Trees Classifier
4. SGD Classifier
5. Linear SVC
6. Gaussian NB
7. KNN

I each model's random_state when appropriate and set the SVM's class_weight to balanced to account for the unbalanced data. There's a lot that can be done in terms of tuning the hyperparameters. I may have to come back to this later to further test different configurations.

In [ ]:
# help(sklearn.ensemble.RandomForestClassifier())
# help(sklearn.naive_bayes.GaussianNB)
help(sklearn.svm.LinearSVC)
# import sklearn
# help(sklearn)

In [ ]:
# We Store prediction of each model in our dict
# Helper Functions for our models. 

def percep(X_train,Y_train,X_test,Y_test,Models):
    perceptron = Perceptron(max_iter = 1000, tol = 0.001, random_state=42)
    perceptron.fit(X_train, Y_train)
    Y_pred = perceptron.predict(X_test)
    Models['Perceptron'] = [accuracy_score(Y_test,Y_pred),confusion_matrix(Y_test,Y_pred)]
    return

def ranfor(X_train,Y_train,X_test,Y_test,Models):
    randomfor = RandomForestClassifier(max_features="sqrt",
                                       n_estimators = 700,
                                       max_depth = None,
                                       n_jobs=-1,
                                       random_state=42
                                      )
    randomfor.fit(X_train,Y_train)
    Y_pred = randomfor.predict(X_test)
    Models['Random Forests'] = [accuracy_score(Y_test,Y_pred),confusion_matrix(Y_test,Y_pred)]
    return

def dec_tree(X_train,Y_train,X_test,Y_test,Models):
    decision_tree = DecisionTreeClassifier(class_weight="balanced",random_state=42)
    decision_tree.fit(X_train, Y_train)
    Y_pred = decision_tree.predict(X_test)
    Models['Decision Tree'] = [accuracy_score(Y_test,Y_pred),confusion_matrix(Y_test,Y_pred)]
    return

def SGDClass(X_train,Y_train,X_test,Y_test,Models):
    sgd = SGDClassifier(max_iter = 1000, tol = 0.001, class_weight = "balanced", random_state=42)
    sgd.fit(X_train, Y_train)
    Y_pred = sgd.predict(X_test)
    Models['SGD Classifier'] = [accuracy_score(Y_test,Y_pred),confusion_matrix(Y_test,Y_pred)]
    return

def linSVC(X_train,Y_train,X_test,Y_test,Models):
    linear_svc = LinearSVC(class_weight="balanced", random_state=42)
    linear_svc.fit(X_train, Y_train)
    Y_pred = linear_svc.predict(X_test)
    Models['SVM'] = [accuracy_score(Y_test,Y_pred),confusion_matrix(Y_test,Y_pred)]
    return

def bayes(X_train,Y_train,X_test,Y_test,Models):
    gaussian = GaussianNB()
    gaussian.fit(X_train, Y_train)
    Y_pred = gaussian.predict(X_test)
    Models['Bayes'] = [accuracy_score(Y_test,Y_pred),confusion_matrix(Y_test,Y_pred)]
    return

def Nearest(X_train,Y_train,X_test,Y_test,Models):
    knn = KNeighborsClassifier(n_neighbors = 3)
    knn.fit(X_train, Y_train)
    Y_pred = knn.predict(X_test)
    Models['KNN'] = [accuracy_score(Y_test,Y_pred),confusion_matrix(Y_test,Y_pred)]

def run_all_and_Plot(df):
    Models = dict()
    from sklearn.model_selection import train_test_split
    X_all = df.drop(['winner'], axis=1)
    y_all = df['winner']
    X_train, X_test, Y_train, Y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=0)
    percep(X_train,Y_train,X_test,Y_test,Models)
    ranfor(X_train,Y_train,X_test,Y_test,Models)
    dec_tree(X_train,Y_train,X_test,Y_test,Models)
    SGDClass(X_train,Y_train,X_test,Y_test,Models)
    linSVC(X_train,Y_train,X_test,Y_test,Models)
    bayes(X_train,Y_train,X_test,Y_test,Models)
    Nearest(X_train,Y_train,X_test,Y_test,Models)
    return Models


def plot_bar(dict):
    labels = tuple(dict.keys())
    y_pos = np.arange(len(labels))
    values = [dict[n][0] for n in dict]
    plt.bar(y_pos, values, align='center', alpha=0.5)
    plt.xticks(y_pos, labels,rotation='vertical')
    plt.ylabel('accuracy')
    plt.title('Accuracy of different models')
    plt.show()


def plot_cm(dict):
    count = 1
    fig = plt.figure(figsize=(10,10))
    for model in dict:
        cm = dict[model][1]
        labels = ['W','L']
        ax = fig.add_subplot(4,4,count)
        cax = ax.matshow(cm)
        plt.title(model,y=-0.8)
        fig.colorbar(cax)
        ax.set_xticklabels([''] + labels)
        ax.set_yticklabels([''] + labels)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        # plt.subplot(2,2,count)
        count+=1
    plt.tight_layout()
    plt.show()

In [ ]:
accuracies = run_all_and_Plot(dropdata)
CompareAll = dict()
CompareAll['Baseline'] = accuracies
for key,val in accuracies.items():
    print(str(key) +' '+ str(val[0]))
plot_bar(accuracies)
plot_cm(accuracies)

Theoretically, we should get best results from our Random Forests Model, thus attempting to tune hyper parameters using GridSearch from Scikit learn

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
#X_all = dropdata.drop(['winner'], axis=1)
#y_all = dropdata['winner']
#X_train, X_test, Y_train, Y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=23)
#rfc = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=50, oob_score = True, max_depth=None) 
#param_grid = { 
#    'n_estimators': [200,700],
#    'max_features': ['auto', 'sqrt', 'log2']
#}

#CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
#CV_rfc.fit(X_train, Y_train)
#print(CV_rfc.best_params_)

### Trying to improve results by dividing features

This block turns each individual red and blue round stat into a ratio of red to blue values. It divides the number of features from 895 to 450. For instance, it turns R_Round4_Strikes_Kicks_Landed and B_Round4_Strikes_Kicks_Landed into a single ratio of red to blue strikes kicks landed. Interesting approach, it appears to be part of th iteration process to see what will be most effective in this analysis.

In [ ]:
dontchange = ['winner','Event_ID','Fight_ID','Max_round','Last_round','B_Age','R_Age']
numeric_cols = [col for col in dropdata if col not in dontchange]
dropdata[numeric_cols] += 1 

In [ ]:
newDF = dropdata.copy()
blue_cols = [col for col in dropdata.columns if 'B__' in col]
red_cols = [col for col in dropdata.columns if 'R__' in col]
for (blue,red) in zip(blue_cols,red_cols):
    newkey = ''.join(str(blue).split('_')[2:])
    dropdata[newkey] = dropdata[str(blue)]/dropdata[str(red)]
    del dropdata[str(blue)]
    del dropdata[str(red)]
newDF.head()

In [ ]:
accuracies = run_all_and_Plot(dropdata)
for key,val in accuracies.items():
    print(str(key) +' '+ str(val[0]))
CompareAll['Blue/Red'] = accuracies
plot_bar(accuracies)
plot_cm(accuracies)


#### Dropping Round 4 and Round 5 since most fights are 3 round Max.

This block drops round 4 and round 5 columns from the dataset but keeps fights that last 5 rounds. It may be an attempt to overcome the amount of null values in the dataset. I'd go a different route since this doesn't seem very precise. I'd tidy and consolidate the data from all rounds into one so there wouldn't be a need to drop round 4 and 5 data. 

In [ ]:
r4 = [col for col in dropdata.columns if "Round4" in col]
r5 = [col for col in dropdata.columns if "Round5" in col]
threerounds = dropdata.drop(r4+r5,axis = 1)
accuracies = run_all_and_Plot(threerounds)
for key,val in accuracies.items():
    print(str(key)+' '+str(val[0]))
CompareAll['DropR4&R5'] = accuracies
plot_bar(accuracies)
plot_cm(accuracies)

#### Dropping 5 round fights entirely

This block drops all title fights. I think this is a mistake for a couple reasons. The dataset is small enough already (only 1477 observations) so dropping any records would probably have a negative impact on the machine learning algorithm. Also, there may be a better way to represent the data that would negate the reasoning behind this step.

In [ ]:
foobar = threerounds.loc[threerounds['Max_round'] == 3]
bewb = threerounds.drop(['Max_round','Last_round'],axis=1)
accuracies = run_all_and_Plot(bewb)
for key,val in accuracies.items():
    print(str(key)+' '+str(val[0]))
CompareAll['Drop5RoundFights'] = accuracies
plot_bar(accuracies)
plot_cm(accuracies)

### Dropping First Fights

This drops any fights in which the fighter has no previous recorded. I may use this in my model but again I'm hesitant to do so because it involves a huge loss of data. Looking at the original CSV, there are 342 records in for Red and 499 records for blue in which they have no previously recorded fight data. I can't afford to lose more than half the data.

This is actually a pretty interesting problem because I'm not sure how to deal with new fighters. By definition, their stats will show up as zeros if they don't have any fight data. It might be worth exploring how to get records prior to 2014 and going from there. Possible sources include:

1. Fightmetrics
2. Fight data from other fight organizations
3. Looking into what the original analyst meant when he said the previous data was not as granular

In [ ]:
data[data.RPrev == False]

In [ ]:
data[data.BPrev == False]

In [ ]:
blahblah = bewb[bewb.Prev != 1]
accuracies = run_all_and_Plot(blahblah)
for key,val in accuracies.items():
    print(str(key)+' '+str(val[0]))
CompareAll['DroppingDebut'] = accuracies
plot_bar(accuracies)
plot_cm(accuracies)

In [ ]:
blue_cols

### Aggregate Round Stats

This block sums the stats for each round into one value. For example, it combines round 1-5 strikes_landed into a single value. This is what I was thinking of doing. May have to borrow this code in my analysis.

In [ ]:
blue_cols
newDF.info()
b_feats = list(set([x[10:] for x in blue_cols if "Round" in x]))
r_feats = list(set([x[10:] for x in red_cols if "Round" in x]))
def sumshit(b_feats,cols):
    for x in b_feats:
        newDF.loc[:,x] = 0
        for y in cols:
            if x in y:
                newDF[x] += newDF[y]
                newDF.drop(y,axis=1,inplace=True)
sumshit(b_feats,blue_cols)
sumshit(r_feats,red_cols)
newDF.info()


In [ ]:
newDF.describe()
accuracies = run_all_and_Plot(newDF)
for key,val in accuracies.items():
    print(str(key) +' '+ str(val[0]))
CompareAll['SumRounds'] = accuracies
plot_bar(accuracies)
plot_cm(accuracies)



### Comparing Red to Blue

This block compares red stats to blue stats. It creates a ratio of strikes landed by red vs strikes landed by blue for each category. Very useful. I think I'm going to borrow this code as well since it's an idea I was thinking about implementing in my reanalysis.

In [ ]:
blue_cols = [col for col in newDF.columns if 'B__' in col]
red_cols = [col for col in newDF.columns if 'R__' in col]
for (blue,red) in zip(blue_cols,red_cols):
    newkey = ''.join(str(blue).split('_')[2:])
    newDF[newkey] = newDF[str(blue)]/newDF[str(red)]
    del newDF[str(blue)]
    del newDF[str(red)]

In [ ]:
accuracies = run_all_and_Plot(newDF)
for key,val in accuracies.items():
    print(str(key) +' '+ str(val[0]))
CompareAll['SumRounds'] = accuracies
plot_bar(accuracies)
plot_cm(accuracies)

### Reducing Features

This block drops features that are seemingly arbitrary and may have little effect on the data. It drops weight, hometown, event location, event id, fight id, max round and last round. Interestingly, the accuracy scores dip slightly after this is done. I'm not sure I agree with the decisions made here. For instance, I still want to take a look at splitting the columns by city and country. I'm not sure about the others. I'll have to combe back to this later down the road.

In [ ]:
reduced_features = newDF.drop(["Weight","B_HomeTown","B_Location", "Event_ID", "Fight_ID", "Max_round", "Last_round", "R_HomeTown", "R_Location"],axis = 1)
accuracies = run_all_and_Plot(reduced_features)
for key,val in accuracies.items():
    print(str(key) +' '+ str(val[0]))
CompareAll['Reduced Features'] = accuracies
plot_bar(accuracies)
plot_cm(accuracies)


In [ ]:
reduced_features.info()

In [ ]:
sorted(reduced_features.columns.values.tolist())

## Conclusion
Our model has a best predictive value that lies between 58-63% on average between runs. Despite a very low accuracy model, we believe this is the best possible given the amount of available data and its inherent noise. 

## Stretch Goals

### Rewrite and re-run scraper to pull data from earlier years

It won't be as granular, but it may strengthen the analysis. There might not be enough records from 2014 onwards to model the data accurately. On the other hand, I'm not even sure the previous analyst modeled the data accurately. He took fight data from each fight and used that to predict the winner. That's not realistic. You don't have a round-by-round analysis before the fight happens! It would be much better to use the running totals to predict a fight's outcome. Looks like we're learning Beautiful Soup! At least you have a starting point.


## Scraper Rewrite

The following scraper retrieves data from ufc.com. It currently pulls a single fighter's data. I added functionality to convert that data into a JSON file, extract the list of fights, and write that to a CSV but it's still incomplete. Still need the following steps:

1. Pull from list of all fighters
2. Merge all fight data so opponent data shows up
3. Flatten data where appropriate
4. Format data appropriately

I'm scrapping this and building my own scraper. The data on fightmetric is more robust. The data I'm getting from ufc.com has a lot of missing data in general and it does not break down each fight as well. It lists strike percentages but does not give raw numbers. The fightmetric data lists raw data as well as percentages and may lead to a more complete picture of each fighter. It also does not require as much merging. Looks like we're starting fresh!

### Fight URL Scraper

This first script pulls the urls of all fights from fightmetric.com and exports them as a CSV.

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib
import os
os.chdir('/Users/courtneyfergusonlee/kaggle-practice/ufc-scraper/MMA-scraper-master')

# Open the main page with events listed
sock = urllib.urlopen('http://www.fightmetric.com/statistics/events/completed?page=all')
page = sock.read()
soup = BeautifulSoup(page)

# Scrape event URLs from the main page
event_urls = []
trs = soup.find_all('tr')
for tr in trs:
    for link in tr.find_all('a'):
        event_urls.append(link.get('href'))


# Pull Fight URLs from each Event URL
fight_urls = []
for event_url in event_urls: 
    print event_url
    try:
        sock = urllib.urlopen(event_url)
        event_html = sock.read()
        event_soup = BeautifulSoup(event_html)

        tds = event_soup.find_all('td')
        for td in tds:
            for link in td.find_all('a'):
                url = link.get('href')
                url_type = url.split('-')[0][-2:] # Fight vs. Fighter; last 2 letters
                if url_type == 'ht': # use er for fighter, ht for fight
                    #print url
                    fight_urls.append(url)
    
    except urllib2.HTTPError:
        print "HTTP Error"
        pass

# Save fight URLs to a csv file
fight_urls = pd.DataFrame(fight_urls, columns=['link'])
fight_urls.to_csv('fight urls.csv', index=False)



### Fight Scraper

This scrapes all fight data from fightmetric.com/fight-details. It reads a list of URLs, pulls the fight statistics from each fight, and concatenates it onto a Pandas dataframe. There are over 4000 urls in the list, so it takes a while to run (over 3 hours).

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib
import os
os.chdir('/Users/courtneyfergusonlee/ufc_fight_analysis/MMA-scraper-master')

# Load URLs from CSV (created in fightmetric_scraper.py)
fight_urls = pd.read_csv('fight urls.csv', encoding='utf-8')['link'].values.tolist()

# Initialize an empty dataframe
fighter_df = pd.DataFrame(columns=['name_first', 'name_last', 'kd', 'sig_strikes', 'sig_attempts', 'strikes', 'strike_attempts', 
                                   'takedowns', 'td_attempts', 'sub_attempts', 'pass', 'reversals', 'head', 'head_attempts', 'body', 
                                   'body_attempts','leg', 'leg_attempts', 'distance', 'distance_attempts', 'clinch', 'fight_id',
                                   'clinch_attempts', 'ground', 'ground_attempts', 'win/loss', 'referee', 'round', 'method'])


# Iterate through the fight urls, and pull relevant variables/fields
for i in range(len(fight_urls)):
    if i==10:
        print i, fight_urls[i]
        break
    
    sock = urllib.urlopen(fight_urls[i]) # specific URL for a fight
    fight_html = sock.read()
    fight_soup = BeautifulSoup(fight_html, "html.parser")
    trs = fight_soup.find_all('tr') # all the tables in each fight URL
    headers = fight_soup.find_all('i')
    bad_call = 0
    try: 
        referee = str(headers[24].get_text()).split()[1] + ' ' + str(headers[24].get_text()).split()[-1]
    except:
        referee = None
    try:
        rounds = str(headers[18].get_text()).split()[1]
    except:
        rounds = None
    try:
        method = str(headers[17].get_text()).split()[0]
    except:
        method = None
    try:
        tr1 = str(trs[1].get_text()).split()
        # Find the location of the 2nd table tr2 (it varies)
        j = 0
        while j < 10:
            if str(trs[j].get_text()).split()[6] == 'Head':
                #print j+1
                tr2 = str(trs[j+1].get_text()).split()
                j = 10
            else:
                j += 1
        #print tr1; #print tr2
        
        # Test for the end of names
        k = 0
        while k < len(tr1):
            try:
                int(tr1[k])
                break
            except:
                k += 1
                continue
        #print k
    except:
        print str(i) + ' bad call' if i%20 == 0 else None
        bad_call += 1
        continue


    # Add each fighter's information to the dataframe
    fighter1 = pd.DataFrame({'name_first': tr1[:1], 'name_last': tr1[1:2], 'kd': tr1[k], 'sig_strikes': tr1[k+2],
    'sig_attempts': tr1[k+4], 'strikes': tr1[k+10], 'strike_attempts': tr1[k+12], 'takedowns': tr1[k+16],'td_attempts': tr1[k+18],
    'sub_attempts': tr1[k+24], 'pass': tr1[k+26], 'reversals': tr1[k+28], 'head': tr2[k+8], 'head_attempts': tr2[k+10],
    'body': tr2[k+14], 'body_attempts': tr2[k+16], 'leg': tr2[k+20], 'leg_attempts': tr2[k+22], 'distance': tr2[k+26],
    'distance_attempts': tr2[k+28], 'clinch': tr2[k+32], 'clinch_attempts': tr2[k+34], 'ground': tr2[k+38], 
    'ground_attempts': tr2[k+40], 'win/loss': 1, 'referee': referee, 'round': rounds, 'method': method, 'fight_id': i})

    fighter2 = pd.DataFrame({'name_first': tr1[2:3], 'name_last': tr1[3:4], 'kd': tr1[k+1], 'sig_strikes': tr1[k+5], 
    'sig_attempts': tr1[k+7], 'strikes': tr1[k+13], 'strike_attempts': tr1[k+15], 'takedowns': tr1[k+19],'td_attempts': tr1[k+21],
    'sub_attempts': tr1[k+25], 'pass': tr1[k+27], 'reversals': tr1[k+29], 'head': tr2[k+11], 'head_attempts': tr2[k+13],
    'body': tr2[k+17], 'body_attempts': tr2[k+19], 'leg': tr2[k+23], 'leg_attempts': tr2[k+25], 'distance': tr2[k+29],
    'distance_attempts': tr2[k+31], 'clinch': tr2[k+35], 'clinch_attempts': tr2[k+37], 'ground': tr2[k+41], 
    'ground_attempts': tr2[k+43], 'win/loss': 0, 'referee': referee, 'round': rounds, 'method': method, 'fight_id': i})
    
    fighter_df = pd.concat([fighter_df, fighter1, fighter2], axis=0, ignore_index=True)
    
